<a href="https://colab.research.google.com/github/edgarriba/kornia-examples/blob/master/kornia_lightning_mnist_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kornia and PyTorch Lightning GPU data augmentation demo

In this tutorial we show how one can combine both Kornia and PyTorch Lightning to perform data augmentation to train a model using CPUs and GPUs in batch mode without additional effort.


Enjoy the example !

## Install Kornia and PyTorch Lightning
Next, we install Kornia and PyTorch Lightning

In [1]:
! pip install git+git://github.com/PyTorchLightning/pytorch-lightning.git@master --upgrade
! pip install kornia -q

  Cloning git://github.com/PyTorchLightning/pytorch-lightning.git (to revision master) to /tmp/pip-req-build-t2r5dn0h
  Running command git clone -q git://github.com/PyTorchLightning/pytorch-lightning.git /tmp/pip-req-build-t2r5dn0h
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 829kB 4.1MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-0.6.1.dev0-cp36-none-any.whl size=148144 sha256=ddf0b060e51c4a4ca30ec9a1f09820e6870b98a648e781a22f0ba1c9b603d345
  Stored in directory: /tmp/pip-ephem-wheel-cache-ft44bq0c/wheels/10/69/8d/aa7539d71fd3f79eec266a45eae3206c704f37fcf4ff976e0d
Successfully built pytorch-lightning
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=38234a7b66dfd3e5ccc036f2ce4970e0a52a45b7a2f5dcfc34dc0c3049685305
  Stored in directory: /root/.cache/p

     |████████████████████████████████| 143kB 2.7MB/s 


## Define lightning model

In [2]:
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

import pytorch_lightning as pl
import kornia as K

class CoolSystem(pl.LightningModule):

    def __init__(self):
        super(CoolSystem, self).__init__()
        # not the best model...
        self.l1 = torch.nn.Linear(28 * 28, 10)

        self.transform = torch.nn.Sequential(
            K.augmentation.RandomRectangleErasing((.05, .1), (.3, 1/.3)),
            K.augmentation.RandomRotation((-15., 15.))
        )
    
        self.pil_to_tensor = lambda x: K.image_to_tensor(x)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        # REQUIRED
        x, y = batch
        x_aug = self.transform(x)  # => we perform GPU/Batched data augmentation
        y_hat = self.forward(x_aug)
        loss = F.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        # OPTIONAL
        x, y = batch
        y_hat = self.forward(x)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}
        
    def test_step(self, batch, batch_idx):
        # OPTIONAL
        x, y = batch
        y_hat = self.forward(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss}
        return {'avg_test_loss': avg_loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        return torch.optim.Adam(self.parameters(), lr=0.0004)

    def prepare_data(self):
        MNIST(os.getcwd(), train=True, download=True, transform=self.pil_to_tensor)
        MNIST(os.getcwd(), train=False, download=True, transform=self.pil_to_tensor)

    def train_dataloader(self):
        # REQUIRED
        dataset = MNIST(os.getcwd(), train=True, download=False, transform=self.pil_to_tensor)
        loader = DataLoader(dataset, batch_size=32)
        return loader

    def val_dataloader(self):
        dataset = MNIST(os.getcwd(), train=True, download=False, transform=self.pil_to_tensor)
        loader = DataLoader(dataset, batch_size=32)
        return loader

    def test_dataloader(self):
        dataset = MNIST(os.getcwd(), train=False, download=False, transform=self.pil_to_tensor)
        loader = DataLoader(dataset, batch_size=32)
        return loader

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/core/decorators.py:13: UserWarning: data_loader decorator deprecated in 0.6.1. Will remove 0.8.0
  warnings.warn(w)


## Run training

In [3]:
from pytorch_lightning import Trainer

model = CoolSystem()

# most basic trainer, uses good defaults
trainer = Trainer()
trainer.fit(model)

INFO:root:
  | Name        | Type                   | Params
---------------------------------------------------
0 | l1          | Linear                 | 7 K   
1 | transform   | Sequential             | 0     
2 | transform.0 | RandomRectangleErasing | 0     
3 | transform.1 | RandomRotation         | 0     


Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw


Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw




Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw


Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/training_io.py:358: UserWarning: Did not find hyperparameters at model.hparams. Saving checkpoint without hyperparameters
  "Did not find hyperparameters at model.hparams. Saving checkpoint without"


1

In [4]:
trainer.test(model)

INFO:root:
  | Name        | Type                   | Params
---------------------------------------------------
0 | l1          | Linear                 | 7 K   
1 | transform   | Sequential             | 0     
2 | transform.0 | RandomRectangleErasing | 0     
3 | transform.1 | RandomRotation         | 0     
INFO:root:Model and Trainer restored from checkpoint: /content/lightning_logs/version_0/checkpoints/_ckpt_epoch_3.ckpt
